# fastlite-migrations

> Simple migrations for fastlite

Manage your database migrations with ease.

## Usage

### Installation

Install from [pypi][pypi]:

```sh
$ pip install fastlite_migrations
```

Or install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/ozpau/fastlite-migrations.git
```

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [pypi][pypi].

[repo]: https://github.com/ozpau/fastlite-migrations
[docs]: https://ozpau.github.io/fastlite-migrations/
[pypi]: https://pypi.org/project/fastlite-migrations/
[conda]: https://anaconda.org/ozpau/fastlite-migrations

## How to use

In [ ]:
from fastlite import *
from fastlite_migrations import *

In [ ]:
db = database(":memory:")

In [ ]:
m = Migrator(db)

In [ ]:
@m.add_migration(0)
def initial_db(db): db.q("CREATE TABLE users (id INTEGER PRIMARY KEY AUTOINCREMENT)")

@m.add_migration(1)
def add_cats(db): db.q("CREATE TABLE cats (id INTEGER PRIMARY KEY AUTOINCREMENT)")

@m.add_migration(2)
def add_cat_hats_prop(db): db.q("ALTER TABLE cats ADD COLUMN hats TEXT")

In [ ]:
m.migrate()

0 initial_db
1 add_cats
2 add_cat_hats_prop


In [ ]:
cats = db.t.cats
Cat = cats.dataclass()

In [ ]:
hat_types = ["Top Hat", "Baseball Cap", "Cowboy Hat", "Wizard Hat", "Beanie", "Pirate Hat",
             "Sun Hat", "Party Hat", "Bunny Ears", "Chef Hat", "Hard Hat", "Feathered Hat",
             "Crown", "Sailor Hat", "Headband with Flowers"]

In [ ]:
import random

In [ ]:
for i in range(100):
    cats.insert(Cat(i, random.choice(hat_types)))

In [ ]:
L(cats())

(#100) [Cats(id=0, hats='Party Hat'),Cats(id=1, hats='Sailor Hat'),Cats(id=2, hats='Crown'),Cats(id=3, hats='Top Hat'),Cats(id=4, hats='Bunny Ears'),Cats(id=5, hats='Feathered Hat'),Cats(id=6, hats='Sun Hat'),Cats(id=7, hats='Crown'),Cats(id=8, hats='Pirate Hat'),Cats(id=9, hats='Headband with Flowers'),Cats(id=10, hats='Bunny Ears'),Cats(id=11, hats='Feathered Hat'),Cats(id=12, hats='Pirate Hat'),Cats(id=13, hats='Chef Hat'),Cats(id=14, hats='Chef Hat'),Cats(id=15, hats='Headband with Flowers'),Cats(id=16, hats='Cowboy Hat'),Cats(id=17, hats='Cowboy Hat'),Cats(id=18, hats='Sun Hat'),Cats(id=19, hats='Pirate Hat')...]

What if we now decide we want to store hat names in a separate table?

In [ ]:
hat_types_zh = ["高顶帽", "棒球帽", "牛仔帽", "巫师帽", "毛线帽", "海盗帽", "太阳帽", "派对帽",
                "兔耳朵", "厨师帽", "安全帽", "羽毛帽", "王冠", "水手帽", "花头带"]
hat_types_fr = ["Chapeau haut de forme", "Casquette de baseball", "Chapeau de cowboy",
                "Chapeau de sorcier", "Bonnet", "Chapeau de pirate", "Chapeau de soleil",
                "Chapeau de fête", "Oreilles de lapin", "Chapeau de chef", "Casque de chantier",
                "Chapeau à plumes", "Couronne", "Chapeau de marin", "Bandeau avec des fleurs"]
hat_prices = [15.99, 10.50, 20.00, 25.00, 8.99, 18.50, 12.00, 5.99, 7.50, 14.99, 22.00, 16.75, 30.00, 11.50, 9.99]

In [ ]:
@m.add_migration(3)
def add_hats_table(db):
    db.q("""
CREATE TABLE hats (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name_en TEXT NOT NULL,
    name_fr TEXT NOT NULL,
    name_zh TEXT NOT NULL,
    price INTEGER NOT NULL
)
    """)

While you work on writing the function above, you can find it convenient to try running migration,
and then cancelling it using the rollback function below:

In [ ]:
@m.add_rollback(3)
def delete_hats_table(db): db.q("DROP TABLE hats")

In [ ]:
m.migrate()

3 add_hats_table


In [ ]:
m.rollback()

3 delete_hats_table


Let's finish writing the migration:

In [ ]:
data = list(map(lambda x: dict(zip(['name_en', 'name_fr', 'name_zh', 'price'], x)), zip(hat_types, hat_types_fr, hat_types_zh, hat_prices)))
data[:2]

[{'name_en': 'Top Hat',
  'name_fr': 'Chapeau haut de forme',
  'name_zh': '高顶帽',
  'price': 15.99},
 {'name_en': 'Baseball Cap',
  'name_fr': 'Casquette de baseball',
  'name_zh': '棒球帽',
  'price': 10.5}]

In [ ]:
@m.add_migration(3)
def add_hats_table(db):
    db.q("""
CREATE TABLE hats (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name_en TEXT NOT NULL,
    name_fr TEXT NOT NULL,
    name_zh TEXT NOT NULL,
    price INTEGER NOT NULL
)
    """)

    db.t.hats.insert_all(data)

In [ ]:
m.rollback()

3 delete_hats_table


In [ ]:
m.migrate()

3 add_hats_table


In [ ]:
db.rollback?

Signature: db.rollback()
Docstring: Roll back a transaction
File:      ~/nbdev/contrib/apswutils/apswutils/db.py
Type:      method

In [ ]:
db.t.hats()

[{'id': 1,
  'name_en': 'Top Hat',
  'name_fr': 'Chapeau haut de forme',
  'name_zh': '高顶帽',
  'price': 15.99},
 {'id': 2,
  'name_en': 'Baseball Cap',
  'name_fr': 'Casquette de baseball',
  'name_zh': '棒球帽',
  'price': 10.5},
 {'id': 3,
  'name_en': 'Cowboy Hat',
  'name_fr': 'Chapeau de cowboy',
  'name_zh': '牛仔帽',
  'price': 20},
 {'id': 4,
  'name_en': 'Wizard Hat',
  'name_fr': 'Chapeau de sorcier',
  'name_zh': '巫师帽',
  'price': 25},
 {'id': 5,
  'name_en': 'Beanie',
  'name_fr': 'Bonnet',
  'name_zh': '毛线帽',
  'price': 8.99},
 {'id': 6,
  'name_en': 'Pirate Hat',
  'name_fr': 'Chapeau de pirate',
  'name_zh': '海盗帽',
  'price': 18.5},
 {'id': 7,
  'name_en': 'Sun Hat',
  'name_fr': 'Chapeau de soleil',
  'name_zh': '太阳帽',
  'price': 12},
 {'id': 8,
  'name_en': 'Party Hat',
  'name_fr': 'Chapeau de fête',
  'name_zh': '派对帽',
  'price': 5.99},
 {'id': 9,
  'name_en': 'Bunny Ears',
  'name_fr': 'Oreilles de lapin',
  'name_zh': '兔耳朵',
  'price': 7.5},
 {'id': 10,
  'name_en': 'Che

## Developer Guide

If you are new to using `nbdev` here are some useful pointers to get you started.

### Install fastlite_migrations in Development mode

```sh
# make sure fastlite_migrations package is installed in development mode
$ pip install -e .

# make changes under nbs/ directory
# ...

# compile to have changes apply to fastlite_migrations
$ nbdev_prepare
```